In [ ]:
# first init

#code here

In [ ]:
# third init
# code here 

In [ ]:
# 작자를 위한 <장르불문 흥행작을 만드는 작법의 원칙>
# 쌓아둔 자료집/도감 이젠 활용하자! 당신을 베스트셀러 작가로 만들어줄 실전 노하우와 훈련법

In [ ]:
# fifth init

In [ ]:
# sixth init